<h1>ДЗ №3. Линейная регрессия</h1>
Предсказываем цену алмазов по параметрам камня.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

In [10]:
df = pd.read_csv('diamonds_train.csv')
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.51,Good,D,SI2,63.9,55.0,1180,5.04,5.10,3.24
1,0.72,Ideal,E,VS2,60.8,57.0,3091,5.79,5.82,3.53
2,0.70,Very Good,D,VVS2,62.8,60.0,4022,5.65,5.69,3.56
3,0.36,Ideal,D,SI1,61.2,57.0,663,4.59,4.63,2.82
4,0.54,Very Good,D,SI1,60.0,59.8,1593,5.30,5.34,3.18


In [11]:
df.describe()

,carat,depth,table,price,x,y,z
count,43018.000000,43018.000000,43018.000000,43018.000000,43018.000000,43018.000000,43018.000000
mean,0.796818,61.749672,57.456648,3929.500442,5.730426,5.734358,3.539536
std,0.471362,1.427265,2.238806,3985.321605,1.116451,1.144571,0.703502
min,0.200000,43.000000,43.000000,326.000000,3.730000,3.710000,1.070000
25%,0.400000,61.000000,56.000000,951.000000,4.720000,4.720000,2.910000
50%,0.700000,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,62.500000,59.000000,5317.750000,6.540000,6.530000,4.030000
max,4.500000,79.000000,95.000000,18823.000000,10.230000,58.900000,31.800000


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43018 entries, 0 to 43017
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    43018 non-null  float64
 1   cut      43018 non-null  object 
 2   color    43018 non-null  object 
 3   clarity  43018 non-null  object 
 4   depth    43018 non-null  float64
 5   table    43018 non-null  float64
 6   price    43018 non-null  int64  
 7   x        43018 non-null  float64
 8   y        43018 non-null  float64
 9   z        43018 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 3.3+ MB


Видим, что пропусков нет, и это хорошо.
Преобразуем категориальные данные.

In [38]:
cathegotial_cols = ['cut', 'clarity', 'color']

print(f"cut: {df['cut'].unique()}\nclarity: {df['clarity'].unique()}\ncolor: {df['color'].unique()}")

for i in cathegotial_cols:
    le = LabelEncoder()
    df[i+'_encoded'] = le.fit_transform(df[i])

df

cut: ['Good' 'Ideal' 'Very Good' 'Premium' 'Fair']
clarity: ['SI2' 'VS2' 'VVS2' 'SI1' 'VS1' 'VVS1' 'IF' 'I1']
color: ['D' 'E' 'J' 'F' 'H' 'I' 'G']


,carat,cut,color,clarity,depth,table,price,x,y,z,cut_encoded,clarity_encoded,color_encoded
0,0.51,Good,D,SI2,63.9,55.0,1180,5.04,5.10,3.24,1,3,0
1,0.72,Ideal,E,VS2,60.8,57.0,3091,5.79,5.82,3.53,2,5,1
2,0.70,Very Good,D,VVS2,62.8,60.0,4022,5.65,5.69,3.56,4,7,0
3,0.36,Ideal,D,SI1,61.2,57.0,663,4.59,4.63,2.82,2,2,0
4,0.54,Very Good,D,SI1,60.0,59.8,1593,5.30,5.34,3.18,4,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43013,1.20,Premium,E,SI2,60.4,58.0,5779,6.95,6.89,4.18,3,3,1
43014,1.01,Good,J,SI1,58.8,57.0,3853,6.57,6.63,3.88,1,2,6
43015,1.01,Ideal,E,VVS2,61.7,55.0,10852,6.44,6.49,3.99,2,7,1
43016,0.39,Ideal,E,SI1,62.0,56.0,676,4.72,4.65,2.91,2,2,1


In [56]:
df[['carat', 'depth', 'table', 'x', 'y', 'z']].corr()

,carat,depth,table,x,y,z
carat,1.000000,0.028703,0.180814,0.978239,0.948832,0.957416
depth,0.028703,1.000000,-0.301835,-0.024398,-0.028521,0.095279
table,0.180814,-0.301835,1.000000,0.195727,0.182640,0.150536
x,0.978239,-0.024398,0.195727,1.000000,0.969065,0.971339
y,0.948832,-0.028521,0.182640,0.969065,1.000000,0.948282
z,0.957416,0.095279,0.150536,0.971339,0.948282,1.000000


Как видно, признаки 'carat, 'x', 'y' и 'z' очень сильно коррелируют. Так как в дальнейшем планируется использоваться линейная регрессия, то в обучающую выборку они не пойдут, т.к. результат работы линейной регрессии страдает от "мультиколлинеарности".

Рискнем здоровьем и попробуем использвать линейную регрессию без нормализации и стандартизации

In [57]:
X = df[['carat', 'depth', 'table', 'cut_encoded', 'clarity_encoded', 'color_encoded']]
Y = df['price']
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

In [62]:
model = LinearRegression()
model.fit(X_train, Y_train)

train_pred = model.predict(X_train)
val_pred = model.predict(X_val)

r2_train = r2_score(Y_train, train_pred)
r2_val = r2_score(Y_val, val_pred)

rmse_train = np.sqrt(mean_squared_error(Y_train, train_pred))
rmse_val = np.sqrt(mean_squared_error(Y_val, val_pred))

print(f"R2:\nTrain: {r2_train}\nValidation: {r2_val}\nRMSE:\nTrain: {rmse_train}\nValidation: {rmse_val}")

R2:
Train: 0.8818670639788315
Validation: 0.8821158303756037
RMSE:
Train: 1366.721930169119
Validation: 1380.2401154561092


Теперь попробуем со всеми переменными

In [ ]:
X1 = df[['carat', 'depth', 'table', 'cut_encoded', 'clarity_encoded', 'color_encoded', 'x', 'y', 'z']]
Y1 = df['price']
X1_train, X1_val, Y1_train, Y1_val = train_test_split(X1, Y1, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X1_train, Y1_train)

train_pred = model.predict(X1_train)
val_pred = model.predict(X1_val)

r2_train = r2_score(Y1_train, train_pred)
r2_val = r2_score(Y_val, val_pred)

rmse_train = np.sqrt(mean_squared_error(Y1_train, train_pred))
rmse_val = np.sqrt(mean_squared_error(Y1_val, val_pred))

print(f"R2:\nTrain: {r2_train}\nValidation: {r2_val}\nRMSE:\nTrain: {rmse_train}\nValidation: {rmse_val}")

R2:
Train: 0.8891521862617
Validation: 0.8048090186939145
RMSE:
Train: 1323.9092913636846
Validation: 1776.0564900528063


Как видим, значения RMSE и R2 на выборке без признаков 'x', 'y' и 'z' дают лучшие результаты, чем с ними, значит предположение оказалось верным

Попробуем нормализовать и стандартизировать признаки

In [ ]:
standart_scaler = StandardScaler()
X_train_standart = standart_scaler.fit_transform(X_train)
X_val_standart = standart_scaler.transform(X_val)

minmax_scaler = MinMaxScaler()
X_train_scaled = minmax_scaler.fit_transform(X_train_standart)
X_val_scaled = minmax_scaler.transform(X_val_standart)

model = LinearRegression()
model.fit(X_train_scaled, Y_train)

train_pred = model.predict(X_train_scaled)
val_pred = model.predict(X_val_scaled)

r2_train = r2_score(Y_train, train_pred)
r2_val = r2_score(Y_val, val_pred)

rmse_train = np.sqrt(mean_squared_error(Y_train, train_pred))
rmse_val = np.sqrt(mean_squared_error(Y_val, val_pred))

print(f"R2:\nTrain: {r2_train}\nValidation: {r2_val}\nRMSE:\nTrain: {rmse_train}\nValidation: {rmse_val}")

R2:
Train: 0.8818670639788315
Validation: 0.8821158303756038
RMSE:
Train: 1366.7219301691193
Validation: 1380.2401154561082


Как видим, результаты между нормализироавнными-стандартизированными данными и данными без изменений схожи.

Дерево решений. Попробуем задать им разные признаки: одной скормим все признаки, другой те, которые независимы. Также попробуем аккуратно ограничить модели от переобучения.

In [122]:
model = DecisionTreeRegressor(max_depth=15, min_samples_split=20) # 2 модели: одна со всеми признаками, другая - без мультиколлинеарных
model1 = DecisionTreeRegressor(max_depth=15, min_samples_split=20)

model.fit(X_train, Y_train)
model1.fit(X1_train, Y1_train)

# Не все признаки
train_pred = model.predict(X_train)
val_pred = model.predict(X_val)

r2_train = r2_score(Y_train, train_pred)
r2_val = r2_score(Y_val, val_pred)

rmse_train = np.sqrt(mean_squared_error(Y_train, train_pred))
rmse_val = np.sqrt(mean_squared_error(Y_val, val_pred))

print(f"No multicollinear features:\nR2:\nTrain: {r2_train}\nValidation: {r2_val}\nRMSE:\nTrain: {rmse_train}\nValidation: {rmse_val}")

# Все признаки
train_pred = model1.predict(X1_train)
val_pred = model1.predict(X1_val)

r2_train = r2_score(Y1_train, train_pred)
r2_val = r2_score(Y_val, val_pred)

rmse_train = np.sqrt(mean_squared_error(Y1_train, train_pred))
rmse_val = np.sqrt(mean_squared_error(Y1_val, val_pred))

print(f"All features\nR2:\nTrain: {r2_train}\nValidation: {r2_val}\nRMSE:\nTrain: {rmse_train}\nValidation: {rmse_val}")

No multicollinear features:
R2:
Train: 0.9867635687676443
Validation: 0.9740073476401087
RMSE:
Train: 457.4884330334114
Validation: 648.115190379121
All features
R2:
Train: 0.988123548444294
Validation: 0.9704632205881524
RMSE:
Train: 433.3492157642371
Validation: 690.8893070065303


Метрики на дереве выросли относительно регрессии, падение на валидации не очень значительно
И попробуем дерево решений на обработанных данных(станд. + норм.)

In [91]:
model = DecisionTreeRegressor(max_depth=15, min_samples_split=20)
model.fit(X_train_scaled, Y_train)

train_pred = model.predict(X_train_scaled)
val_pred = model.predict(X_val_scaled)

r2_train = r2_score(Y_train, train_pred)
r2_val = r2_score(Y_val, val_pred)

rmse_train = np.sqrt(mean_squared_error(Y_train, train_pred))
rmse_val = np.sqrt(mean_squared_error(Y_val, val_pred))

print(f"Decision Tree + standartion-normalization\nR2:\nTrain: {r2_train}\nValidation: {r2_val}\nRMSE:\nTrain: {rmse_train}\nValidation: {rmse_val}")

Decision Tree + standartion-normalization
R2:
Train: 0.9867635687676443
Validation: 0.9740272605709149
RMSE:
Train: 457.4884330334114
Validation: 647.866882790089


Как видим, разницы между нормализованными данными и данными без изменений практически никакой.
Переобучим дерево с такими параметрами на всей тренировочной выборке и получим предсказания по тестовой выборке

In [124]:
final_model = DecisionTreeRegressor(max_depth=15, min_samples_split=20)
final_model.fit(X, Y)

Y_pred = final_model.predict(X)
r2 = r2_score(Y, Y_pred)
rmse = np.sqrt(mean_squared_error(Y, Y_pred))
print(f"R2: {r2}, RMSE: {rmse}")

R2: 0.98712964439103, RMSE: 452.1198751532121


In [125]:
test_df = pd.read_csv('diamonds_test.csv')

test_df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,1.02,Good,F,SI2,59.2,58.0,6.51,6.56,3.87
1,1,0.70,Very Good,I,VVS1,59.5,58.0,5.78,5.81,3.45
2,2,0.32,Very Good,H,VVS2,63.4,56.0,4.37,4.34,2.76
3,3,0.42,Ideal,F,VVS2,62.2,56.0,4.79,4.82,2.99
4,4,0.40,Ideal,F,VS2,62.3,54.0,4.74,4.77,2.96


In [126]:
cathegotial_cols = ['cut', 'clarity', 'color']

for i in cathegotial_cols:
    le = LabelEncoder()
    test_df[i+'_encoded'] = le.fit_transform(test_df[i])

test_df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,cut_encoded,clarity_encoded,color_encoded
0,0,1.02,Good,F,SI2,59.2,58.0,6.51,6.56,3.87,1,3,2
1,1,0.70,Very Good,I,VVS1,59.5,58.0,5.78,5.81,3.45,4,6,5
2,2,0.32,Very Good,H,VVS2,63.4,56.0,4.37,4.34,2.76,4,7,4
3,3,0.42,Ideal,F,VVS2,62.2,56.0,4.79,4.82,2.99,2,7,2
4,4,0.40,Ideal,F,VS2,62.3,54.0,4.74,4.77,2.96,2,5,2


In [127]:
X_test = test_df[['carat', 'depth', 'table', 'cut_encoded', 'clarity_encoded', 'color_encoded']]

Y_final_pred = model.predict(X_test)

result = pd.DataFrame({'id': test_df['id'], 'price': Y_final_pred})
result.to_csv('submission.csv', index=False)